In [1]:
import timeit
import matplotlib.pyplot as plt
import pickle
from mnist import MNIST
from time import time

In [2]:
print("Loading dataset....")
mndata = MNIST('../mnist_dataset')
X_train, y_train = mndata.load_training()
X_train = (mndata.process_images_to_numpy(X_train)/255)
X_test, y_test = mndata.load_testing()
X_test = (mndata.process_images_to_numpy(X_test)/255)
print("Done.")

Loading dataset....
Done.


In [3]:
def one_hot_encode(labels):
    y_inp=np.zeros((len(labels),10))
    for ind,val in enumerate(labels):
        y_inp[ind][val]=1
    return y_inp

In [4]:
def batch_gen(st=0,size=20,validate=False):
    st=st%60000
    if validate:
        X=X_test[st:st+size].reshape(-1,28,28,1)
        labels=y_test[st:st+size]
    else:
        X=X_train[st:st+size].reshape(-1,28,28,1)
        labels=y_train[st:st+size]
    y=one_hot_encode(labels)
    return X,y

In [5]:
from nnet.network import Sequential,layers
from nnet.layers import conv2d,max_pool,flatten,dense,dropout
from nnet import functions
from nnet import optimizers
import numpy as np

Seed: 211
Seed: 262


In [6]:
def generator():
    model=Sequential()
    model.add(dense(256,input_shape=(100),activation=functions.relu))
    model.add(dense(512,activation=functions.relu))
    model.add(dense(1024,activation=functions.relu))
    model.add(dense(784,activation=functions.sigmoid))
    return model
g=generator()

In [7]:
g.summary()

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
0 input_layer(InputLayer) (None, 100)                echo             0
__________________________________________________________________________________________
1 dense(dense)            (None, 256)                relu             25856
__________________________________________________________________________________________
2 dense(dense)            (None, 512)                relu             131584
__________________________________________________________________________________________
3 dense(dense)            (None, 1024)               relu             525312
__________________________________________________________________________________________
4 dense(dense)            (None, 784)                sigmoid          803600
Total Params: 1,486,352
Trainable Params: 1,486,352
Non-trainable Params: 0


In [8]:
def discriminator():
    model=Sequential()
    model.add(dense(1024,input_shape=(784),activation=functions.relu))
    model.add(dropout(0.3))
    model.add(dense(512,activation=functions.relu))
    model.add(dropout(0.3))
    model.add(dense(256,activation=functions.relu))
    model.add(dense(1,activation=functions.sigmoid))
    return model
d=discriminator()

In [9]:
d.summary()

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
0 input_layer(InputLayer) (None, 784)                echo             0
__________________________________________________________________________________________
1 dense(dense)            (None, 1024)               relu             803840
__________________________________________________________________________________________
2 dropout(dropout)        (None, 1024)               echo             0
__________________________________________________________________________________________
3 dense(dense)            (None, 512)                relu             524800
__________________________________________________________________________________________
4 dropout(dropout)        (None, 512)                echo             0
__________________________________________________________________________________________
5 dens

In [10]:
g.compile(optimizer=optimizers.adam,loss=functions.cross_entropy_with_logits,learning_rate=0.01)
d.compile(optimizer=optimizers.adam,loss=functions.cross_entropy_with_logits,learning_rate=0.01)

In [11]:
def train_gan(gan_input,labels):
    x = g.fit(gan_input,labels)
    gan_out = d.predict(x)
    return gan_out

In [15]:
def run():
    batch_size=64
    dsz=X_train.shape[0]
    for _b in range(dsz//batch_size):
        stt=time()
        noise=np.random.normal(0,1,(batch_size,100))
        # Generate fake images from noise
        generated_images=g.predict(noise)
        # Get random real images
        real_images=X_train[np.random.randint(low=0,high=dsz,size=batch_size)]
        X = np.concatenate([real_images,generated_images])
        # Label real ones as 90% real
        y_dis=np.zeros((2*batch_size,1))
        y_dis[:batch_size]=0.99
        # Train discriminator
        d.fit(X,y_dis)
        print("\rProgress: {:.2f} %     Sample time: {:.3f}s    _".format(_b*batch_size*50/dsz,time()-stt),end='')
    for _b in range(dsz//batch_size):
        stt=time()
        # Treat noised input of generator as real data
        noise=np.random.normal(0,1,(batch_size,100))
        y_gen=np.ones((batch_size,1))
        # Train generator
        g.fit(noise,y_gen)
        print("\rProgress: {:.2f} %     Sample time: {:.3f}s    _".format(50+_b*batch_size*50/dsz,time()-stt),end='')

In [ ]:
epochs=50
for epoch in range(epochs):
    print("EPOCH:",epoch+1,'/',epochs)
    st_tm=time()
    run()
    print("Epoch time: {}:{}s".format(int(time()-st_tm)//60,int(time()-st_tm)%60))

In [ ]:
noise=np.random.normal(0,1,(1,100))
plt.imshow(g.predict(noise).reshape(28,28), cmap='Greys')
plt.show()

In [ ]:
g.predict(noise)